In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !numba -s

In [3]:
import pandas as pd
import numpy as np
import torch 
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
cuda = torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [5]:
cuda

True

In [6]:
from network import*
from dataset import*
from loss import*
from train import*

In [7]:
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models

In [8]:
df = pd.read_csv('nawa.csv')

In [9]:
df2 = df.sample(frac=.3).reset_index(drop=True)

In [10]:
df2.head()

,anchor,positive,negative
0,magg4.png,magg13.png,bred5.png
1,cavi19.png,cavi5.png,gddy7.png
2,bred19.png,bred1.png,magg14.png
3,born12.png,born4.png,burb6.png
4,magi4.png,magi16.png,gddy18.png


In [11]:
df2.shape

(205200, 3)

In [12]:
data = MyDataset(df2,'images',size=256,al = True)

In [13]:
dl = DataLoader(data,batch_size=8,shuffle=True)

In [14]:
a = next(iter(dl))

In [15]:
a[0][0].shape

torch.Size([8, 3, 256, 256])

In [16]:
model = TripletNet(models.resnet18(),pretrained=False,fci = 32768)
if cuda:
    model.cuda()

In [17]:
# model

In [18]:
margin  = 1000000
loss_fn = TripletLoss(margin)

In [19]:
lr = 1e-4
optimizer = optim.Adam(model.parameters(),lr = lr)
scheduler =  lr_scheduler.StepLR(optimizer,8,gamma=0.1,last_epoch=-1)
n_epoch = 1
log_interval = 100

In [ ]:
fit(dl,model,loss_fn,optimizer,scheduler,n_epoch,cuda,log_interval)

Train: [0/205200 (0%)]	Loss: 1000003.187500
Train: [800/205200 (0%)]	Loss: 178225.052187
Train: [1600/205200 (1%)]	Loss: 0.000000
Train: [2400/205200 (1%)]	Loss: 0.000000
Train: [3200/205200 (2%)]	Loss: 0.000000
Train: [4000/205200 (2%)]	Loss: 0.000000
Train: [4800/205200 (2%)]	Loss: 0.000000
Train: [5600/205200 (3%)]	Loss: 0.000000
Train: [6400/205200 (3%)]	Loss: 0.000000
Train: [7200/205200 (4%)]	Loss: 0.000000
Train: [8000/205200 (4%)]	Loss: 0.000000
Train: [8800/205200 (4%)]	Loss: 0.000000
Train: [9600/205200 (5%)]	Loss: 0.000000
Train: [10400/205200 (5%)]	Loss: 0.000000
Train: [11200/205200 (5%)]	Loss: 0.000000
Train: [12000/205200 (6%)]	Loss: 0.000000
Train: [12800/205200 (6%)]	Loss: 0.000000
Train: [13600/205200 (7%)]	Loss: 0.000000
Train: [14400/205200 (7%)]	Loss: 0.000000
Train: [15200/205200 (7%)]	Loss: 0.000000
Train: [16000/205200 (8%)]	Loss: 0.000000
Train: [16800/205200 (8%)]	Loss: 0.000000
Train: [17600/205200 (9%)]	Loss: 0.000000
Train: [18400/205200 (9%)]	Loss: 0.00000

In [ ]:
# fit(dl,model,loss_fn,optimizer,scheduler,20,cuda,log_interval)

In [22]:
torch.save(model.state_dict(),'embed')

In [29]:
df2 = pd.read_csv('image_list.csv')

In [38]:
df2.head()

,fn,label
0,born1.png,born
1,born10.png,born
2,born11.png,born
3,born12.png,born
4,born13.png,born


In [31]:
im = load_image('born1.png',256,'images')

In [32]:
im.shape

torch.Size([3, 256, 256])

In [33]:
im = np.reshape(im,(1,3,256,256))

In [39]:
xx = model.GetEmbeddings(im.cuda())

In [47]:
list(xx.cpu().detach().numpy()[0])

[-4990.28,
 7697.1177,
 4169.2476,
 -7514.59,
 6530.62,
 -10390.133,
 -3269.3997,
 6162.5996,
 3017.5276,
 7228.7334,
 9165.038,
 3980.049,
 -7670.0264,
 -9057.381,
 -6774.463,
 9601.697,
 6330.0415,
 -4940.11,
 -7173.067,
 -6752.992,
 -6969.798,
 4233.055,
 2172.0088,
 -11260.571,
 -5197.475,
 7070.6396,
 -2978.1658,
 9170.258,
 -7503.7466,
 8714.46,
 6771.347,
 7306.9043,
 4514.2354,
 -7883.579,
 2186.7021,
 8511.243,
 4296.605,
 -8728.136,
 -12455.781,
 8137.37,
 -4225.087,
 7073.6196,
 -4472.3184,
 -6650.138,
 -5641.061,
 -2403.2705,
 7845.827,
 -7293.2637,
 -4714.912,
 -4340.063,
 -5659.022,
 5852.084,
 3527.475,
 -7879.9507,
 -12480.835,
 5284.689,
 6490.6064,
 8343.513,
 -3443.0684,
 -4737.79,
 5270.7285,
 -8777.927,
 10158.244,
 3279.845]

In [49]:
embeds = []
for i in list(df2['fn']):
    im = load_image(i,256,'images')
    im = np.reshape(im,(1,3,256,256))
    emb = model.GetEmbeddings(im.cuda())
    emb = list(emb.cpu().detach().numpy()[0])
    embeds.append(emb)

In [53]:
embeds = np.array(embeds)

In [55]:
embeds

array([[-4990.28  ,  7697.1177,  4169.2476, ..., -8777.927 , 10158.244 ,
         3279.845 ],
       [-5063.8877,  7810.28  ,  4230.2334, ..., -8906.748 , 10307.338 ,
         3327.5686],
       [-4928.079 ,  7601.184 ,  4116.9165, ..., -8669.059 , 10030.782 ,
         3238.4006],
       ...,
       [-4771.279 ,  7360.005 ,  3986.3801, ..., -8393.43  ,  9712.412 ,
         3136.0112],
       [-4680.5693,  7219.4155,  3910.3354, ..., -8233.158 ,  9527.339 ,
         3076.379 ],
       [-4706.9736,  7260.2495,  3932.3594, ..., -8279.466 ,  9581.018 ,
         3093.5212]], dtype=float32)

In [56]:
embb = pd.DataFrame(embeds)

In [59]:
embb[['fn','label']] = df2[['fn','label']]

In [60]:
embb

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,fn,label
0,-4990.279785,7697.117676,4169.247559,-7514.589844,6530.620117,-10390.132812,-3269.399658,6162.599609,3017.527588,7228.733398,...,6490.606445,8343.512695,-3443.068359,-4737.790039,5270.728516,-8777.926758,10158.244141,3279.844971,born1.png,born
1,-5063.887695,7810.279785,4230.233398,-7624.627930,6626.752930,-10541.802734,-3316.747314,6253.035645,3061.486816,7334.584473,...,6585.634277,8465.495117,-3495.150391,-4807.115234,5348.044434,-8906.748047,10307.337891,3327.568604,born10.png,born
2,-4928.079102,7601.184082,4116.916504,-7420.374512,6449.837891,-10259.719727,-3228.127686,6085.100586,2979.979004,7139.013672,...,6409.104492,8238.780273,-3401.234619,-4677.213379,5204.930664,-8669.058594,10030.782227,3238.400635,born11.png,born
3,-4784.318848,7379.615234,3996.782471,-7204.186035,6262.020020,-9960.328125,-3133.649902,5907.505371,2893.004883,6930.910645,...,6221.963867,7998.325195,-3302.806641,-4540.480957,5053.438477,-8416.890625,9738.260742,3143.603516,born12.png,born
4,-5081.834473,7838.078613,4245.208008,-7651.666992,6650.722168,-10579.057617,-3328.465332,6274.666992,3072.472900,7361.368652,...,6608.875488,8495.560547,-3507.838135,-4823.250488,5367.198730,-8939.047852,10343.342773,3339.142822,born13.png,born
5,-5034.710938,7765.463379,4205.824707,-7580.721680,6589.160156,-10480.992188,-3297.633301,6216.390137,3044.036133,7293.272949,...,6547.556641,8416.827148,-3475.232910,-4778.306641,5317.466797,-8856.312500,10247.340820,3308.204834,born14.png,born
6,-4825.716797,7443.757812,4031.425781,-7266.513184,6315.842285,-10046.191406,-3160.893311,5958.747559,2917.768555,6990.689941,...,6275.878418,8067.437500,-3331.482910,-4579.839844,5097.118164,-8489.536133,9822.378906,3170.876953,born15.png,born
7,-4873.877930,7517.779785,4071.576660,-7338.775391,6378.786621,-10146.399414,-3192.407227,6018.032715,2946.927246,7060.714844,...,6338.506348,8148.032227,-3364.284668,-4625.543945,5147.779297,-8573.971680,9920.150391,3202.645020,born16.png,born
8,-4788.328613,7385.606934,4000.237793,-7209.992676,6267.188965,-9968.983398,-3136.781494,5912.940430,2895.593994,6936.812012,...,6227.350098,8005.179199,-3304.528076,-4544.671387,5057.294922,-8423.470703,9746.543945,3146.612061,born17.png,born
9,-4963.608398,7656.349609,4146.660156,-7474.017090,6496.322754,-10333.381836,-3251.285645,6129.009277,3001.242432,7190.821289,...,6455.295410,8298.175781,-3426.153809,-4710.786621,5242.624023,-8732.015625,10103.003906,3261.643799,born18.png,born


In [61]:
embb.to_csv('embeddings.csv',index = False)

In [67]:
np.array(embb.iloc[0][:-2],dtype = np.float)

array([ -4990.27978516,   7697.11767578,   4169.24755859,  -7514.58984375,
         6530.62011719, -10390.1328125 ,  -3269.3996582 ,   6162.59960938,
         3017.52758789,   7228.73339844,   9165.03808594,   3980.04907227,
        -7670.02636719,  -9057.38085938,  -6774.46289062,   9601.69726562,
         6330.04150391,  -4940.10986328,  -7173.06689453,  -6752.9921875 ,
        -6969.79785156,   4233.05517578,   2172.00878906, -11260.57128906,
        -5197.47509766,   7070.63964844,  -2978.16577148,   9170.2578125 ,
        -7503.74658203,   8714.45996094,   6771.34716797,   7306.90429688,
         4514.23535156,  -7883.57910156,   2186.70214844,   8511.24316406,
         4296.60498047,  -8728.13574219, -12455.78125   ,   8137.37011719,
        -4225.08691406,   7073.61962891,  -4472.31835938,  -6650.13818359,
        -5641.06103516,  -2403.27050781,   7845.82714844,  -7293.26367188,
        -4714.91210938,  -4340.06298828,  -5659.02197266,   5852.08398438,
         3527.47509766,  

In [85]:
imt = load_image('test.jpg',256,'test')
# im = load_image(imt,256,'images')
im = np.reshape(im,(1,3,256,256))
emb = model.GetEmbeddings(im.cuda())
emb = list(emb.cpu().detach().numpy()[0])


In [88]:
np.array(embb)[189][:-2]-emb

array([-190.66796875, 294.228515625, 159.362548828125, -287.1083984375,
       250.00244140625, -397.1748046875, -124.763671875, 235.0830078125,
       115.4189453125, 276.9609375, 349.873046875, 152.20068359375,
       -293.20166015625, -346.09765625, -258.91259765625, 366.7685546875,
       242.119140625, -188.87158203125, -274.3525390625, -258.65966796875,
       -266.50244140625, 161.97509765625, 83.45166015625, -429.9814453125,
       -198.4755859375, 270.4033203125, -113.798095703125, 349.87890625,
       -286.52783203125, 332.7138671875, 258.78955078125, 279.43798828125,
       173.00244140625, -301.11669921875, 83.653076171875, 325.283203125,
       164.501220703125, -333.36328125, -475.86328125, 310.986328125,
       -161.59619140625, 269.9609375, -170.783203125, -254.79931640625,
       -215.23095703125, -92.09716796875, 299.970703125, -278.1669921875,
       -179.974609375, -166.205810546875, -216.28173828125, 223.38671875,
       135.282470703125, -301.2998046875, -477.0839

In [78]:
npembb = np.array(embb)
losses= []
names = []
for i in range(200):
    loss = np.sum(np.power((npembb[i][:-2]-np.array(emb)),2))
    losses.append(loss)
    names.append(npembb[i][-2])

In [79]:
losses

[9840419.944612563,
 15583241.944768727,
 5983293.000710666,
 733902.6814772487,
 17185846.654295146,
 13134111.873327553,
 1728113.2078204155,
 3407221.1325134635,
 810213.3946454525,
 8063030.629550397,
 1161976.2889493704,
 18533300.796089053,
 364611.5533192158,
 21397508.84072715,
 6102395.667390108,
 4283022.258572221,
 14552177.044664085,
 282769.4069869518,
 469654.75520314276,
 2435695.672831893,
 1605695.315846473,
 7935964.780607015,
 16039251.189277664,
 16352166.243724167,
 9266570.475741655,
 10733274.96114558,
 12345512.281667292,
 6445774.052337602,
 6743549.370782912,
 5890653.091324046,
 6024233.815590158,
 2093769.90306063,
 1455232.8351300359,
 341742.608262077,
 183346.5023431927,
 3297606.832512766,
 8613125.628736377,
 13669.884650230408,
 649156.1158471107,
 558425.9641336948,
 10704333.64838761,
 341869.4092903137,
 582.5252003669739,
 806864.8643375039,
 14887260.59196116,
 123977.19913953543,
 11122032.709927097,
 949383.4146888852,
 628647.3861558437,
 34511

In [80]:
names

['born1.png',
 'born10.png',
 'born11.png',
 'born12.png',
 'born13.png',
 'born14.png',
 'born15.png',
 'born16.png',
 'born17.png',
 'born18.png',
 'born19.png',
 'born2.png',
 'born20.png',
 'born3.png',
 'born4.png',
 'born5.png',
 'born6.png',
 'born7.png',
 'born8.png',
 'born9.png',
 'bred1.png',
 'bred10.png',
 'bred11.png',
 'bred12.png',
 'bred13.png',
 'bred14.png',
 'bred15.png',
 'bred16.png',
 'bred17.png',
 'bred18.png',
 'bred19.png',
 'bred2.png',
 'bred20.png',
 'bred3.png',
 'bred4.png',
 'bred5.png',
 'bred6.png',
 'bred7.png',
 'bred8.png',
 'bred9.png',
 'burb1.png',
 'burb10.png',
 'burb11.png',
 'burb12.png',
 'burb13.png',
 'burb14.png',
 'burb15.png',
 'burb16.png',
 'burb17.png',
 'burb18.png',
 'burb19.png',
 'burb2.png',
 'burb20.png',
 'burb3.png',
 'burb4.png',
 'burb5.png',
 'burb6.png',
 'burb7.png',
 'burb8.png',
 'burb9.png',
 'cavi1.png',
 'cavi10.png',
 'cavi11.png',
 'cavi12.png',
 'cavi13.png',
 'cavi14.png',
 'cavi15.png',
 'cavi16.png',
 'cavi17